In [62]:
import numpy as np

class TreeNode:
    def __init__(self, feature=-1, label=None, left=None, right=None, value=None):
        '''
        Args:
            feature: int #Index of the feature used for splitting the node
            label: str #The class label if the node is a leaf
            left: TreeNode #Left child
            right: TreeNode #Right child
        '''
        self.feature = feature
        self.label = label
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeClassifier:
    def __init__(self, max_depth=11, min_samples_split=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    def compute_entropy(self, y):
        '''
        Input: y: ndarray #Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)
        Output: entropy: flaot #Entropy at that node
        '''
        entropy = 0
        if len(y) > 0:
            p1 = len(y[y==1]) / len(y)
            p2 = (len(y) - p1) / len(y)
            if p1 == 0 or p1 == 1:
                entropy = p1
            else:
                entropy = -(p1 * np.log2(p1) + p2 * np.log2(p2))

        return entropy
    
    def split_dataset(self, X, active_nodes, feature):
        """
        Splits the data at the given node into
        left and right branches
        
        Args:
            X: ndarray    #Data matrix of shape(n_samples, n_features)
            node_indices: list #List containing the active indices. I.e, the samples being considered at this step.
            feature: int  #Index of feature to split on
        
        Returns:
            left_nodes: list     Indices with feature value == 1
            right_nodes: list    Indices with feature value == 0
        """
        left_nodes = []
        right_nodes = []
        for i in active_nodes:
            if X[i][feature] == 1:
                left_nodes.append(i)
            else:
                right_nodes.append(i)

        return left_nodes, right_nodes
    
    def compute_information_gain(self, X, y, active_nodes, feature):
        """
        Compute the information of splitting the node on a given feature
        
        Args:
            X: ndarray   Data matrix of shape(n_samples, n_features)
            y: ndarray   list or ndarray with n_samples containing the target variable
            active_nodes: list  List containing the active indices. I.e, the samples being considered in this step.
            feature: int         Index of feature to split on
    
        Returns:
            information_gain: float       computed information gain if we use feature as a node
        """ 
        left_nodes, right_nodes = self.split_dataset(X, active_nodes, feature)

        X_node, y_node = X[active_nodes], y[active_nodes]
        X_left, y_left = X[left_nodes], y[left_nodes]
        X_right, y_right = X[right_nodes], y[right_nodes]
        
        w_left = len(X_left) / len(X_node) 
        w_right = len(X_right) / len(X_node) 

        h_node = self.compute_entropy(y_node)
        h_left = self.compute_entropy(y_left)
        h_right = self.compute_entropy(y_right)

        weighted_entropy = w_left * h_left + w_right * h_right
        info_gain = h_node - weighted_entropy
        return info_gain
    
    def get_best_split(self, X, y, active_nodes):
        """
        Returns the optimal feature and threshold value
        to split the node data 
        
        Args:
            X: ndarray           Data matrix of shape(n_samples, n_features)
            y: ndarray         list or ndarray with n_samples containing the target variable
            active_nodes: list List containing the active indices. I.e, the samples being considered in this step.

        Returns:
            best_feature: int     The index of the best feature to split
        """  
        best_feature = -1
        max_info_gain = 0
        for feature in range(self.n_features):
            info_gain = self.compute_information_gain(X, y, active_nodes, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature

        return best_feature
    
    def build_tree(self, X, y, active_nodes, depth):
        '''
        Build the decision tree
        Args: 
            X: ndarray #Data matrix of shape(n_samples, n_features)
            y: ndarray #list or ndarray with n_samples containing the target variable
            active_nodes: list #List containing the active indices. I.e, the samples being considered in this step.
            depth: int #Depth of the decision tree
        Returns: 
            root: TreeNode #Root node of the decision tree
        '''
        # Stop early if the active nodes is empty
        if len(active_nodes) == 0:
            return
        # Stop early if the depth is reached
        if depth > self.max_depth:
            return
        # Stop early if the node is pure
        if len(active_nodes) <= self.min_samples_split:
            return

        # Get the best feature
        best_feature = self.get_best_split(X, y, active_nodes)
        # Create the root node
        root = TreeNode(best_feature, f'Node - {best_feature}', None, None, None)
        left, right = self.split_dataset(X, active_nodes, best_feature)
        root.left = self.build_tree(X, y, left, depth+1)
        root.right = self.build_tree(X, y, right, depth+1)
        if root.left is None or root.right is None:
            root.value = y[active_nodes[0]]

        return root
        
    def fit(self, X, y):
        '''
        Fit the decision tree
        Args: 
            X: ndarray #Data matrix of shape(n_samples, n_features)
            y: ndarray #list or ndarray with n_samples containing the target variable
        '''
        self.n_features = X.shape[1]
        active_nodes = list(range(X.shape[0]))
        self.root = self.build_tree(X, y, active_nodes, depth=0)
    
    def predict(self, X):
        '''
        Predict the class labels
        Args: 
            X: ndarray #Data matrix of shape(n_samples, n_features)
        Returns: 
            y_pred: list #List containing the predicted class labels
        '''
        if self.root is None:
            return 'No tree here!'

        y_pred = []
        for i in range(X.shape[0]):
            node = self.root
            while node.value is None:
                if X[i][node.feature] == 1:
                    node = node.left
                else:
                    node = node.right
            y_pred.append(node.value)
        return y_pred

In [59]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
score = np.sum(y_pred == y_train) / len(y_train)
print(score)

0.8


In [61]:
X_train = np.array([[1,1],[0,0]])
y_train = np.array([1,0])
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
score = np.sum(y_pred == y_train) / len(y_train)
print(score)

No tree here!
0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=39a136b1-8191-420f-afab-bb238316f4d7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>